In [5]:
import pandas
import os
import numpy as np


df = pandas.read_csv("1335.vdb.tab", sep="\t", header =None) 
#Récupère la liste de nombre, correspondant aux comptage des différents espèces en fonction des lieu de prélevement
#2=door_in_1 ; 3=door_in_2 ; 4=faucet_handle_1 ; 5=faucet_handle_2 ; 6=sink_floor_1 ; 7=sink_floor_2
#8=soap_dispenser_1 ; 9=stall_in_1 ; 10=toilet_floor_1 ; 11=toilet_floor_1 ; 12=toilet_flush_handle_1
#13=toilet_flush_handle_2 ; 14=toilet_seat_1 ; 15=toilet_seat_2
def getcountsfromrow(row):
    l_l = [ int(row[2]),
            int(row[3]),
            int(row[4]),
            int(row[5]),
            int(row[6]),
            int(row[7]),
            int(row[8]),
            int(row[9]),
            int(row[10]),
            int(row[11]),
            int(row[12]),
            int(row[13]),
            int(row[14]),
            int(row[15]) ]
    return l_l

"""
Code pour generer dictionnaire. 

Au prealable, avec cat 1335.vdb.tab | cut -f 2 | cut -d ";" -f 1 | uniq
OTUConTaxonomy"Bacteria(100)  => on a confirmé que ce sont toutes Bacteria
"""
def getDicoTaxo(df):
    dicotaxo = { "name" : "Bacteria", "count" : [0 for i in range(14)], "children" : []}
    for index,row in df.iterrows():
        leveltax = row[1].split(';')  # uniquement indexes >=1 m'interessent 
        if index > 0:
            COUNTS = getcountsfromrow(row)
            tmp5 = {}
            tmp4 = {}
            tmp3 = {}
            tmp2 = {}
            tmp1 = {}
            tmp5["name"] = leveltax[5]
            tmp4["name"] = leveltax[4]
            tmp3["name"] = leveltax[3]
            tmp2["name"] = leveltax[2]
            tmp1["name"] = leveltax[1]      
            tmp5["count"] =  getcountsfromrow(row)
            tmp5["children"] = []  # lowest level
            tmp4["count"] =  getcountsfromrow(row)
            tmp4["children"] = [tmp5]
            tmp3["count"] =  getcountsfromrow(row)
            tmp3["children"] = [tmp4]
            tmp2["count"] =  getcountsfromrow(row)
            tmp2["children"] = [tmp3]
            tmp1["count"] = getcountsfromrow(row)
            tmp1["children"] = [tmp2]
            tmpcounts = dicotaxo["count"]
            dicotaxo["count"] = np.add(tmpcounts, COUNTS)
            if leveltax[1] not in [m["name"] for m in dicotaxo["children"]] :
                dicotaxo["children"].append(tmp1)
            else:
                for m in dicotaxo["children"]:
                    if m["name"] == leveltax[1]:
                        tmpc = m["count"]
                        m["count"] = np.add(tmpc, COUNTS)
                        if leveltax[2] not in [n["name"] for n in m["children"]] :
                            m["children"].append(tmp2)
                        else:
                            for n in m["children"]:
                                if n["name"] == leveltax[2]:
                                    tmpc = n["count"]
                                    n["count"] = np.add(tmpc, COUNTS)
                                    if leveltax[3] not in [o["name"] for o in n["children"]]:
                                        n["children"].append(tmp3)
                                    else: 
                                        for o in n["children"]:
                                            if o["name"] == leveltax[3]:
                                                tmpc = o["count"]
                                                o["count"] = np.add(tmpc,COUNTS)
                                                if leveltax[4] not in [p["name"] for p in o["children"]]:
                                                    o["children"].append(tmp4)
                                                else:
                                                    for p in o["children"]:
                                                        if p["name"] == leveltax[4]:
                                                            tmpc = p["count"]
                                                            p["count"] = np.add(tmpc, COUNTS)
                                                            if leveltax[5] not in [q["name"] for q in p["children"]]:
                                                                p["children"].append(tmp5)
                                                            else:
                                                                for q in p["children"]:
                                                                    if q["name"] == leveltax[5]:
                                                                        tmpc = q["count"]
                                                                        q["count"] = np.add(tmpc, COUNTS)
                                                                        q["children"] = []
                                                                        # this is the lowest level                                        
    return dicotaxo


In [ ]:
"""
Veut pouvoir récuperer donnée intéressantes dans dictionnaire.
"""

def convertNiv(niveau):
    if niveau.upper() == "REGNE":
        return 1
    if niveau.upper() == "PHYLUM":
        return 2
    if niveau.upper() == "CLASSE":
        return 3
    if niveau.upper() == "ORDRE":
        return 4
    if niveau.upper() == "FAMILLE":
        return 5
    if niveau.upper() == "GENRE":
        return 6
    
#Méthode 1 : En fonction du niveaux de taxo récupéré donnne pour l'espèce le comptage en fonction du lieu
#Input : niveau taxo Regne;Phylum;Classe;Ordre;Famille;Genre
#Output : liste de vecteurs 1er : vecteur des espèces, 
#puis 1 vecteurs par localisation (1 vecteur donne le comptage pour l'espèce dans la localisation donnée)
def getNiv(niveau,dicoTaxo):
    niv = convertNiv(niveau)
    vecteurs = []
    #Regne 
    listeRegne = []
    #Regne compte
    listelRegne = []
    #Autre niveau
    listePhylum =[]
    listeClasse =[]
    listeOrdre =[]
    listeFamille =[]
    listeGenre =[]
    #autre niveau compte
    for i in range(1,15):
        nomList = f"listel{i}" + niveau.lower()
        exec(nomList+'=[]')

    for k1, v1 in dicoTaxo.items():
        #Récupère que pour le règne
        if k1 == "name" and niv == 1:
            listeRegne.append(v1)
            vecteurs.append(listeRegne)
        elif k1 == "count" and niv == 1:
            listelRegne.append(v1)
            vecteurs.append(listelRegne)
        for phylum in dicoTaxo['children']:
            for k2, v2 in phylum.items():
                #Récupère que pour le phylum
                if k2 == "name" and niv == 2:
                    listePhylum.append(v2)
                elif k2 == "count" and niv == 2:
                    listel1phylum.append(v2[0])
                    listel2phylum.append(v2[1])
                    listel3phylum.append(v2[2])
                    listel4phylum.append(v2[3])
                    listel5phylum.append(v2[4])
                    listel6phylum.append(v2[5])
                    listel7phylum.append(v2[6])
                    listel8phylum.append(v2[7])
                    listel9phylum.append(v2[8])
                    listel10phylum.append(v2[9])
                    listel11phylum.append(v2[10])
                    listel12phylum.append(v2[11])
                    listel13phylum.append(v2[12])
                    listel14phylum.append(v2[13])
                #Descend dans les niveau plus précis
                if k2 == "children" and niv > 2:
                    for classe in v2:
                        for k3, v3 in classe.items():
                            #Récupère que pour la classe
                            if k3 == "name" and niv == 3:
                                listeClasse.append(v3)
                            elif k3 == "count" and niv == 3:
                                listel1classe.append(v3[0])
                                listel2classe.append(v3[1])
                                listel3classe.append(v3[2])
                                listel4classe.append(v3[3])
                                listel5classe.append(v3[4])
                                listel6classe.append(v3[5])
                                listel7classe.append(v3[6])
                                listel8classe.append(v3[7])
                                listel9classe.append(v3[8])
                                listel10classe.append(v3[9])
                                listel11classe.append(v3[10])
                                listel12classe.append(v3[11])
                                listel13classe.append(v3[12])
                                listel14classe.append(v3[13])
                            #Descend dans les niveau plus précis
                            elif k3 == "children" and niv > 3:
                                for ordre in v3:
                                    for k4, v4 in ordre.items():
                                        #Récupère que pour l'ordre
                                        if k4 == "name" and niv == 4:
                                            listeOrdre.append(v4)
                                        elif k4 == "count" and niv == 4:
                                            listel1ordre.append(v4[0])
                                            listel2ordre.append(v4[1])
                                            listel3ordre.append(v4[2])
                                            listel4ordre.append(v4[3])
                                            listel5ordre.append(v4[4])
                                            listel6ordre.append(v4[5])
                                            listel7ordre.append(v4[6])
                                            listel8ordre.append(v4[7])
                                            listel9ordre.append(v4[8])
                                            listel10ordre.append(v4[9])
                                            listel11ordre.append(v4[10])
                                            listel12ordre.append(v4[11])
                                            listel13ordre.append(v4[12])
                                            listel14ordre.append(v4[13])
                                        #Descend dans les niveau plus précis
                                        elif k4 == "children" and niv > 4:
                                            for famille in v4:
                                                for k5, v5 in famille.items():
                                                    #Récupère que pour la famille
                                                    if k5 == "name" and niv == 5:
                                                        listeFamille.append(v5)
                                                    elif k5 == "count" and niv == 5:
                                                        listel1famille.append(v5[0])
                                                        listel2famille.append(v5[1])
                                                        listel3famille.append(v5[2])
                                                        listel4famille.append(v5[3])
                                                        listel5famille.append(v5[4])
                                                        listel6famille.append(v5[5])
                                                        listel7famille.append(v5[6])
                                                        listel8famille.append(v5[7])
                                                        listel9famille.append(v5[8])
                                                        listel10famille.append(v5[9])
                                                        listel11famille.append(v5[10])
                                                        listel12famille.append(v5[11])
                                                        listel13famille.append(v5[12])
                                                        listel14famille.append(v5[13])
                                                    #Descend dans les niveau plus précis
                                                    elif k5 == "children" and niv > 5:
                                                        for genre in v5:
                                                            for k6, v6 in genre.items():
                                                                #Récupère que pour la famille
                                                                if k6 == "name" and niv == 6:
                                                                    listeGenre.append(v6)
                                                                elif k6 == "count" and niv == 6:
                                                                    listel1genre.append(v6[0])
                                                                    listel2genre.append(v6[1])
                                                                    listel3genre.append(v6[2])
                                                                    listel4genre.append(v6[3])
                                                                    listel5genre.append(v6[4])
                                                                    listel6genre.append(v6[5])
                                                                    listel7genre.append(v6[6])
                                                                    listel8genre.append(v6[7])
                                                                    listel9genre.append(v6[8])
                                                                    listel10genre.append(v6[9])
                                                                    listel11genre.append(v6[10])
                                                                    listel12genre.append(v6[11])
                                                                    listel13genre.append(v6[12])
                                                                    listel14genre.append(v6[13])


    if niv == 2:
        vecteurs.append(listePhylum);vecteurs.append(listel1phylum);vecteurs.append(listel2phylum);vecteurs.append(listel3phylum)
        vecteurs.append(listel4phylum);vecteurs.append(listel5phylum);vecteurs.append(listel6phylum);vecteurs.append(listel7phylum)
        vecteurs.append(listel8phylum);vecteurs.append(listel9phylum);vecteurs.append(listel10phylum);vecteurs.append(listel11phylum)
        vecteurs.append(listel12phylum);vecteurs.append(listel13phylum);vecteurs.append(listel14phylum);
    if niv == 3:
        vecteurs.append(listeClasse);vecteurs.append(listel1classe);vecteurs.append(listel2classe);vecteurs.append(listel3classe)
        vecteurs.append(listel4classe);vecteurs.append(listel5classe);vecteurs.append(listel6classe);vecteurs.append(listel7classe)
        vecteurs.append(listel8classe);vecteurs.append(listel9classe);vecteurs.append(listel10classe);vecteurs.append(listel11classe)
        vecteurs.append(listel12classe);vecteurs.append(listel13classe);vecteurs.append(listel14classe);
    if niv == 4:
        vecteurs.append(listeOrdre);vecteurs.append(listel1ordre);vecteurs.append(listel2ordre);vecteurs.append(listel3ordre)
        vecteurs.append(listel4ordre);vecteurs.append(listel5ordre);vecteurs.append(listel6ordre);vecteurs.append(listel7ordre)
        vecteurs.append(listel8ordre);vecteurs.append(listel9ordre);vecteurs.append(listel10ordre);vecteurs.append(listel11ordre)
        vecteurs.append(listel12ordre);vecteurs.append(listel13ordre);vecteurs.append(listel14ordre);
    if niv == 5:
        vecteurs.append(listeFamille);vecteurs.append(listel1famille);vecteurs.append(listel2famille);vecteurs.append(listel3famille)
        vecteurs.append(listel4famille);vecteurs.append(listel5famille);vecteurs.append(listel6famille);vecteurs.append(listel7famille)
        vecteurs.append(listel8famille);vecteurs.append(listel9famille);vecteurs.append(listel10famille);vecteurs.append(listel11famille)
        vecteurs.append(listel12famille);vecteurs.append(listel13famille);vecteurs.append(listel14famille);
    if niv == 6:
        vecteurs.append(listeGenre);vecteurs.append(listel1genre);vecteurs.append(listel2genre);vecteurs.append(listel3genre)
        vecteurs.append(listel4genre);vecteurs.append(listel5genre);vecteurs.append(listel6genre);vecteurs.append(listel7genre)
        vecteurs.append(listel8genre);vecteurs.append(listel9genre);vecteurs.append(listel10genre);vecteurs.append(listel11genre)
        vecteurs.append(listel12genre);vecteurs.append(listel13genre);vecteurs.append(listel14genre);
    
    return vecteurs


In [ ]:
import matplotlib.pyplot as plt

##Test plotBare 
##Doit revoire echelle
##aussi trie peut être sur espece pas vraiment représenté. Ou normalisation ? 
dicoTaxoTest = getDicoTaxo(df)
vecteurs = getNiv("Phylum",dicoTaxoTest)

phylums=vecteurs[0]
loc1=vecteurs[1]
loc2=vecteurs[2]
loc3=vecteurs[3]
N=len(phylums)
ind = np.arange(N)
fig = plt.figure()
p1 = plt.bar(ind, loc1, width, color='r')
p2 = plt.bar(ind, loc2, width,bottom=loc1, color='b')
p3 = plt.bar(ind, loc3, width,bottom=loc2, color='g')
plt.ylabel('Abundance') 
plt.title('Cool') 
plt.xticks(ind, (phylums)) 

plt.legend((p1[0], p2[0], p3[0]), ('loc1', 'loc2', 'loc3')) 
  
plt.show() 

In [3]:
import requests
import pandas
import matplotlib.pyplot as plt
import numpy as np

df = pandas.read_csv("1335.vdb.tab", sep="\t", header =None) 
somme = 0
for index,row in tb.iterrows():
    
    if index >0 : 
        somme = int(row[6]) + somme
somme
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,toilet_floor_1,toilet_floor_2,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2
1,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,45,87,13,329,76,156
2,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,430,707,0,25,258,0
3,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,6,30,3,283,41,1059
4,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,236,251,9,51,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4101,Otu04101,Bacteria(100);Bacteroidetes(100);Cytophagia(10...,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4102,Otu04102,Bacteria(100);Proteobacteria(100);Alphaproteob...,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4103,Otu04103,Bacteria(100);Firmicutes(100);unclassified(100...,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4104,Otu04104,Bacteria(100);Acidobacteria(100);Acidobacteria...,0,0,0,0,0,1,0,0,0,0,0,0,0,0
